
### Before starting make sure to convert the semi-columns in the provided csv files to commas.

In [1]:
import math
import numpy as np
import pandas as pd
from typing import Dict, List
from datamodel import Listing, OrderDepth, Trade, TradingState, Order

In [19]:

# prices-1 not prices1 in reality
prices0 = pd.read_csv('prices_round_3_day_0.csv')
prices1 = pd.read_csv('prices_round_3_day_1.csv')
prices2 = pd.read_csv('prices_round_3_day_2.csv')

prices = pd.concat([prices0, prices1, prices2], ignore_index=True)

# trades-1 not trades1 in reality
trades0 = pd.read_csv('trades_round_3_day_0_nn.csv')
trades1 = pd.read_csv('trades_round_3_day_1_nn.csv')
trades2 = pd.read_csv('trades_round_3_day_2_nn.csv')


trades0['day'] = 0
trades1['day'] = 1
trades2['day'] = 2

trades = pd.concat([trades0, trades1, trades2], ignore_index=True)

In [20]:
prices

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,0,0,DIVING_GEAR,99999.0,6.0,99998.0,19.0,NaN,NaN,100001.0,6.0,100002.0,19.0,NaN,NaN,100000.0,0.0
1,0,0,PEARLS,9998.0,1.0,9995.0,30.0,NaN,NaN,10005.0,30.0,NaN,NaN,NaN,NaN,10001.5,0.0
2,0,0,COCONUTS,8113.0,205.0,NaN,NaN,NaN,NaN,8116.0,205.0,NaN,NaN,NaN,NaN,8114.5,0.0
3,0,0,BERRIES,3997.0,19.0,3996.0,58.0,NaN,NaN,4003.0,19.0,4005.0,58.0,NaN,NaN,4000.0,0.0
4,0,0,BANANAS,4893.0,1.0,4892.0,30.0,NaN,NaN,4899.0,30.0,NaN,NaN,NaN,NaN,4896.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,2,999900,COCONUTS,7910.0,212.0,NaN,NaN,NaN,NaN,7913.0,212.0,NaN,NaN,NaN,NaN,7911.5,70584.0
209996,2,999900,DIVING_GEAR,100178.0,28.0,NaN,NaN,NaN,NaN,100181.0,28.0,NaN,NaN,NaN,NaN,100179.5,-510408.0
209997,2,999900,BERRIES,3905.0,19.0,3904.0,55.0,NaN,NaN,3912.0,74.0,NaN,NaN,NaN,NaN,3908.5,-85040.0
209998,2,999900,DOLPHIN_SIGHTINGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3094.0,0.0


In [25]:
trades

,timestamp,buyer,seller,symbol,currency,price,quantity,day
0,0,NaN,NaN,BERRIES,SEASHELLS,4003.0,6,0
1,100,NaN,NaN,PEARLS,SEASHELLS,10002.0,1,0
2,100,NaN,NaN,DIVING_GEAR,SEASHELLS,99999.0,1,0
3,300,NaN,NaN,PEARLS,SEASHELLS,9998.0,2,0
4,300,NaN,NaN,PEARLS,SEASHELLS,9998.0,1,0
...,...,...,...,...,...,...,...,...
21404,999400,NaN,NaN,BANANAS,SEASHELLS,4798.0,1,2
21405,999400,NaN,NaN,BERRIES,SEASHELLS,3911.0,14,2
21406,999400,NaN,NaN,PEARLS,SEASHELLS,10004.0,1,2
21407,999400,NaN,NaN,PEARLS,SEASHELLS,10004.0,1,2


In [23]:
prices_product = {}
trades_product = {}

for product in prices['product'].unique().tolist():    
    string = product.split('_')[0].lower()
    prices_product[product] = 'prices_' + string
    trades_product[product] = 'trades_' + string

In [26]:
prices_dfs = {}
trades_dfs = {}

for product in prices['product'].unique().tolist():
    prices_dfs[prices_product[product]] = prices[prices['product'] == product]
    trades_dfs[trades_product[product]] = trades[trades['symbol'] == product]

In [39]:
prices_product.keys()

dict_keys(['DIVING_GEAR', 'PEARLS', 'COCONUTS', 'BERRIES', 'BANANAS', 'DOLPHIN_SIGHTINGS', 'PINA_COLADAS'])

In [38]:
prices_dfs.keys()

dict_keys(['prices_diving', 'prices_pearls', 'prices_coconuts', 'prices_berries', 'prices_bananas', 'prices_dolphin', 'prices_pina'])

In [29]:
prices_dfs['prices_berries']

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
3,0,0,BERRIES,3997.0,19.0,3996.0,58.0,NaN,NaN,4003.0,19.0,4005.0,58.0,NaN,NaN,4000.0,0.000000
8,0,100,BERRIES,3996.0,12.0,3995.0,59.0,NaN,NaN,4003.0,12.0,4004.0,59.0,NaN,NaN,3999.5,20.396484
18,0,200,BERRIES,3995.0,14.0,3994.0,52.0,NaN,NaN,4002.0,14.0,4003.0,52.0,NaN,NaN,3998.5,26.556641
26,0,300,BERRIES,3995.0,11.0,3994.0,42.0,NaN,NaN,4002.0,11.0,4003.0,42.0,NaN,NaN,3998.5,24.740234
29,0,400,BERRIES,3996.0,13.0,3995.0,56.0,NaN,NaN,4003.0,13.0,4004.0,56.0,NaN,NaN,3999.5,22.498047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209970,2,999500,BERRIES,3904.0,61.0,NaN,NaN,NaN,NaN,3907.0,15.0,3911.0,20.0,3912.0,41.0,3905.5,-84316.000000
209972,2,999600,BERRIES,3904.0,14.0,3903.0,41.0,NaN,NaN,3911.0,14.0,3912.0,41.0,NaN,NaN,3907.5,-83436.000000
209985,2,999700,BERRIES,3904.0,13.0,3903.0,50.0,NaN,NaN,3911.0,13.0,3912.0,50.0,NaN,NaN,3907.5,-83340.000000
209987,2,999800,BERRIES,3904.0,18.0,3903.0,53.0,NaN,NaN,3911.0,18.0,3912.0,53.0,NaN,NaN,3907.5,-82590.000000


In [30]:
# defining type hints(data types of corresponding variables)

Time = int
Symbol = str
Product = str
Position = int
UserId = str
Observation = int
Price = int
Quantity = int

trading_states: List[TradingState] = [None] * 10000
own_trades: Dict[Symbol, List[Trade]] = {}
list_own_trades: List[Dict[Symbol, List[Trade]]] = [{}] * 10000
list_market_trades: List[Dict[Symbol, List[Trade]]] = [{}] * 10000
position: Dict[Product, Position] = {}
listings = {"PRODUCT1": Listing(symbol="PRODUCT1", product="PRODUCT1", denomination="SEASHELLS"),
            "PRODUCT2": Listing(symbol="PRODUCT2", product="PRODUCT2", denomination="SEASHELLS")}

In [31]:
def generate_order_depth_prices_dict(df_row):
    buy_order_depth_keys = []
    buy_order_depth_values = []

    sell_order_depth_keys = []
    sell_order_depth_values = []

    for col in df_row.columns.tolist():
        if 'price' in col:
            if 'bid' in col:
                buy_order_depth_keys.append(col)
            else:
                sell_order_depth_keys.append(col)
        elif 'volume' in col:
            if 'bid' in col:
                buy_order_depth_values.append(col)
            else:
                sell_order_depth_values.append(col)
        else:
            pass

    buy_final = {}
    sell_final = {}

    for key, val in zip(buy_order_depth_keys, buy_order_depth_values):
        buy_final[df_row[key].tolist()[0]] = df_row[val].tolist()[0]

    for key, val in zip(sell_order_depth_keys, sell_order_depth_values):
        sell_final[df_row[key].tolist()[0]] = df_row[val].tolist()[0]

    return (buy_final, sell_final)


In [32]:
list_order_depths: List[Dict[Symbol, List[Trade]]] = [{}] * (100000 * 3)

In [77]:
cnt = 0
start = 0
end = 2


for day in range(start, end + 1, 1):
    
    for timestamp in range(0,10000*100, 100):
        
        order_depths = {}
        
        for product in prices['product'].unique().tolist():
            
            order_depths[product] = OrderDepth()
            
            product_df = prices_dfs[prices_product[product]]
            product_df_row = product_df[product_df['timestamp'] == timestamp]
            product_df_row = product_df_row[product_df_row['day'] == day]
            
            order_depths[product].buy_orders, order_depths[product].sell_orders = generate_order_depth_prices_dict(
                product_df_row)
        
        list_order_depths[cnt * 10000 + int(timestamp/100)] = order_depths
        
    cnt += 1

list_order_depths            

[{'DIVING_GEAR': <datamodel.OrderDepth at 0x2010a7056f0>,
  'PEARLS': <datamodel.OrderDepth at 0x20106f9ca60>,
  'COCONUTS': <datamodel.OrderDepth at 0x2010a78aef0>,
  'BERRIES': <datamodel.OrderDepth at 0x2010a789690>,
  'BANANAS': <datamodel.OrderDepth at 0x2010a78ae90>,
  'DOLPHIN_SIGHTINGS': <datamodel.OrderDepth at 0x2010a704d60>,
  'PINA_COLADAS': <datamodel.OrderDepth at 0x2010a78a8c0>},
 {'DIVING_GEAR': <datamodel.OrderDepth at 0x2017be13ee0>,
  'PEARLS': <datamodel.OrderDepth at 0x20161d612d0>,
  'COCONUTS': <datamodel.OrderDepth at 0x2016108db40>,
  'BERRIES': <datamodel.OrderDepth at 0x20107f63e50>,
  'BANANAS': <datamodel.OrderDepth at 0x20107f076a0>,
  'DOLPHIN_SIGHTINGS': <datamodel.OrderDepth at 0x20107f621a0>,
  'PINA_COLADAS': <datamodel.OrderDepth at 0x2010a7040d0>},
 {'DIVING_GEAR': <datamodel.OrderDepth at 0x2017be13b20>,
  'PEARLS': <datamodel.OrderDepth at 0x201762fbdc0>,
  'COCONUTS': <datamodel.OrderDepth at 0x20107f61030>,
  'BERRIES': <datamodel.OrderDepth at 

In [14]:
# loop version is too slow

# cnt = 0

# for day in range(0, 3, 1):

#     for timestamp in range(0, 10000 * 100, 100):

#         bananas = prices_bananas[prices_bananas['timestamp'] == timestamp]
#         bananas = bananas[bananas['day'] == day]

#         pearls = prices_pearls[prices_pearls['timestamp'] == timestamp]
#         pearls = pearls[pearls['day'] == day]
        
#         coladas = prices_coladas[prices_coladas['timestamp'] == timestamp]
#         coladas = coladas[coladas['day'] == day]
        
#         coconuts = prices_coconuts[prices_coconuts['timestamp'] == timestamp]
#         coconuts = coconuts[coconuts['day'] == day]
        
#         diving = prices_diving[prices_diving['timestamp'] == timestamp]
#         diving = diving[diving['day'] == day]
        
#         berries = prices_berries[prices_berries['timestamp'] == timestamp]
#         berries = berries[berries['day'] == day]

#         order_depths = {"BANANAS": OrderDepth(), "PEARLS": OrderDepth(), 'PINA_COLADAS': OrderDepth(), 'COCONUTS': OrderDepth(), 'DIVING_GEAR': OrderDepth(), 'BERRIES': OrderDepth()}

#         order_depths['BANANAS'].buy_orders, order_depths['BANANAS'].sell_orders = generate_order_depth_prices_dict(
#             bananas)
#         order_depths['PEARLS'].buy_orders, order_depths['PEARLS'].sell_orders = generate_order_depth_prices_dict(
#             pearls)
#         order_depths['PINA_COLADAS'].buy_orders, order_depths['PINA_COLADAS'].sell_orders = generate_order_depth_prices_dict(
#             coladas)
#         order_depths['COCONUTS'].buy_orders, order_depths['COCONUTS'].sell_orders = generate_order_depth_prices_dict(
#             coconuts)
#         order_depths['DIVING_GEAR'].buy_orders, order_depths['DIVING_GEAR'].sell_orders = generate_order_depth_prices_dict(
#             diving)
#         order_depths['BERRIES'].buy_orders, order_depths['BERRIES'].sell_orders = generate_order_depth_prices_dict(
#             berries)
        

#         list_order_depths[cnt * 10000 + int(timestamp/100)] = order_depths

#     cnt += 1
# list_order_depths

[{'BANANAS': <datamodel.OrderDepth at 0x2016bee8b80>,
  'PEARLS': <datamodel.OrderDepth at 0x2016bee8b20>,
  'PINA_COLADAS': <datamodel.OrderDepth at 0x2016bee9720>,
  'COCONUTS': <datamodel.OrderDepth at 0x2016bee96c0>,
  'DIVING_GEAR': <datamodel.OrderDepth at 0x2016bee9780>,
  'BERRIES': <datamodel.OrderDepth at 0x2016bee8c10>},
 {'BANANAS': <datamodel.OrderDepth at 0x201633403d0>,
  'PEARLS': <datamodel.OrderDepth at 0x201634c79a0>,
  'PINA_COLADAS': <datamodel.OrderDepth at 0x20163288610>,
  'COCONUTS': <datamodel.OrderDepth at 0x20163288970>,
  'DIVING_GEAR': <datamodel.OrderDepth at 0x2016328a320>,
  'BERRIES': <datamodel.OrderDepth at 0x201632890c0>},
 {'BANANAS': <datamodel.OrderDepth at 0x201633421d0>,
  'PEARLS': <datamodel.OrderDepth at 0x201632a7ca0>,
  'PINA_COLADAS': <datamodel.OrderDepth at 0x201632a5750>,
  'COCONUTS': <datamodel.OrderDepth at 0x20161d61bd0>,
  'DIVING_GEAR': <datamodel.OrderDepth at 0x20161d61810>,
  'BERRIES': <datamodel.OrderDepth at 0x20161d62230>}

In [54]:
list_order_depths[19999]['DIVING_GEAR'].buy_orders

{99195.0: 9.0, 99194.0: 15.0, nan: nan}

In [90]:
def generate_market_trade_info(df_row):
    symbol = df_row['symbol'].tolist()[0]
    price = df_row['price'].tolist()[0]
    quantity = df_row['quantity'].tolist()[0]
    buyer = df_row['buyer'].tolist()[0]
    seller = df_row['seller'].tolist()[0]

    return (symbol, price, quantity, buyer, seller)


list_market_trades: List[Dict[Symbol, List[Trade]]] = [{}] * (10000 * 3)
list_own_trades: List[Dict[Symbol, List[Trade]]] = [{}] * (10000 * 3)

In [ ]:
cnt = 0

for day in range(-1, 2, 1):

    for timestamp in range(0, 10000 * 100, 100):

        bananas = trades_bananas[trades_bananas['timestamp'] == timestamp]
        bananas = bananas[bananas['day'] == day]

        pearls = trades_pearls[trades_pearls['timestamp'] == timestamp]
        pearls = pearls[pearls['day'] == day]

        coladas = trades_coladas[trades_coladas['timestamp'] == timestamp]
        coladas = coladas[coladas['day'] == day]

        coconut = trades_coconuts[trades_coconuts['timestamp'] == timestamp]
        coconut = coconut[coconut['day'] == day]

        market_trades = {}
        own_trades = {}

        if bananas.shape[0] != 0:
            if ((bananas.buyer.tolist()[0] != "SUBMISSION") and (bananas.seller.tolist()[0] != "SUBMISSION")):
                symbol, price, quantity, buyer, seller = generate_market_trade_info(
                    bananas)
                market_trades['BANANAS'] = Trade(
                    symbol, price, quantity, buyer, seller)

            else:
                symbol, price, quantity, buyer, seller = generate_market_trade_info(
                    bananas)
                own_trades['BANANAS'] = Trade(
                    symbol, price, quantity, buyer, seller)

        if pearls.shape[0] != 0:
            if ((pearls.buyer.tolist()[0] != "SUBMISSION") and (pearls.seller.tolist()[0] != "SUBMISSION")):
                symbol, price, quantity, buyer, seller = generate_market_trade_info(
                    pearls)
                market_trades['PEARLS'] = Trade(
                    symbol, price, quantity, buyer, seller)

            else:
                symbol, price, quantity, buyer, seller = generate_market_trade_info(
                    pearls)
                own_trades['PEARLS'] = Trade(
                    symbol, price, quantity, buyer, seller)

        if coladas.shape[0] != 0:
            if ((coladas.buyer.tolist()[0] != "SUBMISSION") and (coladas.seller.tolist()[0] != "SUBMISSION")):
                symbol, price, quantity, buyer, seller = generate_market_trade_info(
                    coladas)
                market_trades['PINA_COLADAS'] = Trade(
                    symbol, price, quantity, buyer, seller)

            else:
                symbol, price, quantity, buyer, seller = generate_market_trade_info(
                    coladas)
                own_trades['PINA_COLADAS'] = Trade(
                    symbol, price, quantity, buyer, seller)

        if coconut.shape[0] != 0:
            if ((coconut.buyer.tolist()[0] != "SUBMISSION") and (coconut.seller.tolist()[0] != "SUBMISSION")):
                symbol, price, quantity, buyer, seller = generate_market_trade_info(
                    coconut)
                market_trades['COCONUTS'] = Trade(
                    symbol, price, quantity, buyer, seller)

            else:
                symbol, price, quantity, buyer, seller = generate_market_trade_info(
                    coconut)
                own_trades['COCONUTS'] = Trade(
                    symbol, price, quantity, buyer, seller)

        list_market_trades[cnt * 10000 + int(timestamp/100)] = market_trades
        list_own_trades[cnt * 10000 + int(timestamp/100)] = own_trades

    cnt += 1


In [84]:
int(trades.timestamp.max()/100) + 2*10000

29999

In [91]:
cnt = 0
start = 0
end = 2

for day in range(start, end + 1, 1):
    for timestamp in range(0, 10000 * 100, 100):

        market_trades = {}
        own_trades = {}

        for product in trades['symbol'].unique().tolist():

            product_df = trades_dfs[trades_product[product]]
            product_df_row = product_df[product_df['timestamp'] == timestamp]
            product_df_row = product_df_row[product_df_row['day'] == day]
            
            if product_df_row.shape[0] != 0:
                symbol, price, quantity, buyer, seller = generate_market_trade_info(product_df_row)
                
                if ((product_df_row.buyer.tolist()[0] != "SUBMISSION") and (product_df_row.seller.tolist()[0] != "SUBMISSION")):
                    market_trades[product] = Trade(symbol, price, quantity, buyer, seller)
                else:
                    own_trades[product] = Trade(symbol, price, quantity, buyer, seller)

        list_market_trades[cnt * 10000 + int(timestamp/100)] = market_trades
        list_own_trades[cnt * 10000 + int(timestamp/100)] = own_trades

    cnt += 1


In [94]:
list_market_trades[1].

AttributeError: 'dict' object has no attribute 'buyer'

In [95]:
non_empty_indexes = []
for index, ele in enumerate(list_market_trades):
    if ele != {}:
        non_empty_indexes.append(index)

print(len(non_empty_indexes))

13980


In [96]:
assets = trades['symbol'].unique().tolist()

In [100]:
assets

['BERRIES', 'PEARLS', 'DIVING_GEAR', 'BANANAS', 'COCONUTS', 'PINA_COLADAS']

In [97]:
def check_asset_type(market_trade):
    for asset in assets:
        try:
            market_trade[asset].symbol
            break
        except Exception as e:
            pass
    return (market_trade[asset].symbol, market_trade[asset].price, market_trade[asset].quantity)

In [103]:
non_empty_indexes

[0,
 1,
 3,
 4,
 5,
 7,
 12,
 14,
 15,
 17,
 22,
 24,
 25,
 27,
 28,
 29,
 31,
 33,
 37,
 40,
 42,
 43,
 44,
 47,
 48,
 49,
 51,
 54,
 62,
 63,
 66,
 67,
 68,
 69,
 72,
 73,
 74,
 76,
 77,
 78,
 79,
 82,
 83,
 84,
 86,
 87,
 89,
 96,
 103,
 105,
 106,
 108,
 111,
 112,
 113,
 114,
 119,
 120,
 121,
 123,
 125,
 129,
 137,
 140,
 143,
 145,
 148,
 150,
 151,
 154,
 155,
 158,
 160,
 163,
 167,
 168,
 171,
 173,
 178,
 182,
 186,
 189,
 190,
 196,
 197,
 201,
 203,
 206,
 208,
 209,
 211,
 213,
 215,
 216,
 218,
 219,
 220,
 221,
 224,
 225,
 227,
 229,
 230,
 231,
 232,
 236,
 238,
 239,
 240,
 242,
 243,
 245,
 247,
 251,
 252,
 253,
 254,
 260,
 264,
 265,
 266,
 269,
 272,
 274,
 275,
 276,
 277,
 279,
 280,
 281,
 283,
 284,
 285,
 286,
 287,
 290,
 292,
 293,
 294,
 295,
 298,
 299,
 300,
 301,
 306,
 307,
 312,
 315,
 317,
 319,
 320,
 322,
 323,
 324,
 327,
 328,
 329,
 336,
 337,
 338,
 339,
 341,
 342,
 343,
 344,
 346,
 347,
 348,
 349,
 355,
 356,
 357,
 358,
 359,
 360,
 363

In [104]:
a, b, c = check_asset_type(list_market_trades[33])
print(a, b, c)

PEARLS 9996.0 2


### Implementing Trader class from here